In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [3]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

#from glove import Corpus, Glove

from gensim.models import Word2Vec

In [4]:
from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [5]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/simple_20200601/Tagged/stop_clean.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [6]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 '3|CD',
 '4|CD',
 ':|:',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'The|DT',
 'This|DT',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'be|VB',
 'born|VBN',
 'by|IN',
 'can|MD',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'people|NNS',
 'that|IN',
 'that|WDT',
 'their|PRP$',
 'they|PRP',
 'the|DT',
 'to|IN',
 'to|TO',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'with|IN'}

In [7]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/simple_20200601/Tagged/ngram_eval_clean.pkl')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(Grădina|NNP, Zoologică|NNP)",20,-605.961128,2,1
1,"(Ving|NNP, Rhames|NNP)",20,-605.961128,2,1
2,"(Karlovy|NNP, Vary|NNP)",20,-605.961128,2,1
3,"(Gharb|NNP-Chrarda|NNP-Beni|NNP, Hssen|NNP)",20,-605.961128,2,1
4,"(waystations|NNS, shuku|NN-eki|NN)",20,-607.368914,2,1
...,...,...,...,...,...
149995,"(supporters|NNS, of|IN, the|DT)",47,-3262.258611,3,4
149996,"(episodes|NNS, and|CC)",64,-3262.274573,2,3
149997,"(as|IN, old|JJ, as|IN)",48,-3262.394255,3,-1
149998,"(John|NNP, Kerry|NNP)",77,-3262.495991,2,1


In [8]:
min_freq = 20
eval_count = 150000

batch_count = max(ngram_eval.batch)

In [9]:
ngram_eval.batch.value_counts()

 1     47463
 2     33035
 3     24270
 4     16305
 5     10058
 6      7289
 7      4847
 8      2491
 9      1465
 10      942
-2       814
 11      457
-1       186
 12      171
 13      118
 14       65
 15       24
Name: batch, dtype: int64

In [10]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [11]:
def mwe_score(exp, model, stats_frame):
    
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    #oldn = []
    #newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(400)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in batch_model.wv.vocab:

        mwv = model.wv[mwetoken]

        for w in exp:
            if w in model.wv.vocab:
                cvs.append(model.wv[w])

                #oldn.append(batch_model.wv.most_similar(w, topn=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.wv[w], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                #oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        #newn = batch_model.wv.most_similar(mwetoken, topn=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan
        cs = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        #'base_nearest': oldn,
        #'mwe_nearest' : newn,
    }, ignore_index=True)

In [12]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Load model
    print('Loading w2v model')

    batch_model = Word2Vec.load(datapath+'/Models/1 w2v/Tagged/simp_tagged_clean_batch{}.model'.format(bb+1))
    
    print('Gathering MWE stats')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim']) # , 'base_nearest', 'mwe_nearest'
    batch_len = len(batch_dfs[bb].ngram)
    if batch_len >= 20000: 
        printer = 5000
    elif batch_len >= 5000: 
        printer = 1000
    else:
        printer = 200
        
    _i = 0
    
    for exp in batch_dfs[bb].ngram:
        if _i % printer == 0:
            print(' MWE '+str(_i)+'/'+str(batch_len)+': '+'+'.join(exp))
        _i += 1
        
        statsf = mwe_score(exp, batch_model, statsf)
        
    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    

Processing batch 1 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/47463: Grădina|NNP+Zoologică|NNP
 MWE 5000/47463: Dutch|NNP+actress|NN
 MWE 10000/47463: front|JJ+teeth|NNS
 MWE 15000/47463: 2006/07|CD+Segunda|NNP
 MWE 20000/47463: both|DT+directions|NNS
 MWE 25000/47463: East|NNP+End|NN
 MWE 30000/47463: a|DT+living|VBG+thing|NN
 MWE 35000/47463: as|IN+commander|NN
 MWE 40000/47463: the|DT+invading|VBG
 MWE 45000/47463: Edward|NNP+IV|NNP
Processing batch 2 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/33035: Senateacting|NNP+asGovernor|NNP
 MWE 5000/33035: Energy|NNP+is|VBZ
 MWE 10000/33035: their|PRP$+computers|NNS
 MWE 15000/33035: gas|NN+was|VBD
 MWE 20000/33035: it|PRP+means|VBZ+to|TO
 MWE 25000/33035: Age|NN+of|IN+the|DT
 MWE 30000/33035: theory|NN+is|VBZ+a|DT
Processing batch 3 of 15
Loading w2v model
Gathering MWE stats
 MWE 0/24270: Ilhwa|NNP+Chunma|NNP
 MWE 5000/24270: Have|VBP+You|PRP+Ever|RB
 MWE 10000/24270: the|DT+Red|JJ
 MWE 15000/24270: between|IN+1980|CD+and|

In [13]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    print('Appending batch {} of '.format(d+1)+str(batch_count))
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Tagged/Results/simp_tagged_clean_output_001.csv', index=False)

Appending batch 1 of 15
Appending batch 2 of 15
Appending batch 3 of 15
Appending batch 4 of 15
Appending batch 5 of 15
Appending batch 6 of 15
Appending batch 7 of 15
Appending batch 8 of 15
Appending batch 9 of 15
Appending batch 10 of 15
Appending batch 11 of 15
Appending batch 12 of 15
Appending batch 13 of 15
Appending batch 14 of 15
Appending batch 15 of 15


In [14]:
all_batches

,ngram,freq,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim
0,"(was|VBD, born|VBN, here|RB)",32,-2115.954533,3,7,"[1, 1, 0]","[0.13005322, -0.0035864385, 0.06179243, 0.0041...","[[-0.5004486, -0.5991892, -0.7888268, 0.548910...","[nan, nan, -0.26227063]",-0.262271
1,"(to|IN, found|VBN)",54,-2767.761057,2,7,"[1, 0]","[-0.12694843, -0.36336574, 0.006913207, -0.103...","[[-1.3017458, -0.18562949, 0.7762788, -0.00783...","[nan, -0.16069299]",-0.160693
2,"(:|:, end|VB)",23,-1078.890289,2,2,"[1, 0]","[-0.004778812, -0.07245265, -0.030748673, 0.13...","[[-0.29579192, -0.5646692, 0.25214067, 0.59772...","[nan, -0.15938467]",-0.159385
3,"(born|VBN, in|IN, then|RB)",26,-1826.089851,3,3,"[1, 1, 0]","[0.12961091, -0.11182376, 0.025480177, 0.12456...","[[1.6495386, 0.88697046, -0.10751744, -0.18025...","[nan, nan, -0.15382023]",-0.153820
4,"(is|VBZ, both|CC, the|DT)",35,-2521.230138,3,11,"[1, 0, 1]","[0.07370354, -0.02085001, 0.0160751, 0.1190704...","[[-0.063559085, -0.33317208, -0.14216833, 1.21...","[nan, -0.14895083, nan]",-0.148951
...,...,...,...,...,...,...,...,...,...,...
149995,"(of|IN, Chaffin|NNP's, Farm|NNP)",58,-3167.096834,3,9,"[1, -1, 0]","[0.08349732, 0.009498704, -0.1335339, 0.100189...","[[-0.45105252, 0.16672108, 0.04918785, -0.3164...","[nan, nan, 0.23580538]",NaN
149996,"(NHL|NNP, Plus|NNP/Minus|NNP, Award|NNP)",38,-1973.585423,3,10,"[0, -1, 0]","[0.2911135, -0.031326327, -0.11115643, 0.06510...","[[1.0337293, 1.1556895, -0.19845621, 2.1994393...","[0.14594129, nan, 0.28706568]",NaN
149997,"(the|DT, Goblet|NNP, of|IN)",40,-2656.494897,3,10,"[1, -1, 1]","[0.32959798, -0.095800444, -0.000788789, 0.312...","[[-0.6403279, 0.068406515, -0.37032217, -0.525...","[nan, nan, nan]",NaN
149998,"(state|NN, state|NN, state|NN)",48,-2965.054834,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN


In [ ]:
#type(all_batches.stopwords[0])

In [15]:
# RuntimeWarning: Mean of empty slice
#  Appear to be taking the np.nanmean() of a list of NaNs

def inval(inlist,val=0):
    if type(inlist) != list:
        return False
    return val in inlist

#nozero = all_batches[~all_batches.stopwords.apply(inval)]

#minone = nozero[nozero.stopwords.apply(inval,val=-1)]

#minone[minone.stopwords.apply(inval,val=1)]

In [16]:
simp_res_light = all_batches.drop(columns=['mwe_vector', 'component_vectors'])

In [17]:
simp_res_light.to_csv(datapath+'/Models/1 w2v/Tagged/Results/simp_tagged_clean_light_001.csv', index=False)